In [62]:
import os
import pandas as pd
import seaborn as sns

In [2]:
data_dir = '/media/dmlab/My Passport/DATA/sec'
data_filepath = os.path.join(data_dir, '10K_parsed.csv')

In [275]:
df = pd.read_csv(data_filepath)
print(len(df))
df.head()

1429


,form_type,CIK,filing_date,period_of_report,texts_only,image_urls,files_url,form_url,Ticker,CompanyName,item1_business,item1a_risk,item7_mda
0,10-K,66740,2022-02-09,2021-12-31,mmm-202112310000066740falsefalse2021FYP10YP20Y...,[],https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,3M,Item 1. Business.3M Company was incorporated i...,Item 1A. Risk FactorsProvided below is a cauti...,Item 7 may involve risks and uncertainties tha...
1,10-K,66740,2021-02-04,2020-12-31,"Common Stock, Par Value $.01 Per Share--12-312...",[],https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,3M,Item 1. Business.3M Company was incorporated i...,Item 1A. Risk FactorsProvided below is a cauti...,Item 7 may involve risks and uncertainties tha...
2,10-K,66740,2020-02-06,2019-12-31,"Common Stock, Par Value $.01 Per SharetrueP20Y...",[],https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,3M,Item 1. Business.3M Company was incorporated i...,Item 1A. Risk FactorsProvided below is a cauti...,Item 7 may involve risks and uncertainties tha...
3,10-K,91142,2022-02-11,2021-12-31,aos-20211231false0000091142FY2021P3YP3YP5YP1YP...,['https://www.sec.gov/Archives/edgar/data/9114...,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...,AOS,A. O. Smith,ITEM 1 BUSINESSAs used in this annual report ...,Something went wrong!,ITEM 7 MANAGEMENTS DISCUSSION AND ANALYSIS OF...
4,10-K,91142,2021-02-12,2020-12-31,aos-20201231false0000091142FYAOS2020P3YP3YP5YP...,['https://www.sec.gov/Archives/edgar/data/9114...,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...,AOS,A. O. Smith,ITEM 1 BUSINESSOur company is comprised of tw...,Something went wrong!,ITEM 7 MANAGEMENTS DISCUSSION AND ANALYSIS OF...


In [276]:
colname = 'len_item1'
df[colname] = df['item1_business'].apply(lambda x: len(x))
df.sort_values(by=[colname], ascending=False, inplace=True) # item1 길이가 긴 것이 앞에 오도록 정렬
original_len = len(df)

# 두 번에 걸쳐 공시가 발행된 경우, item1의 길이가 긴 문서만 남기고 삭제
df.drop_duplicates(['CIK', 'period_of_report'], keep='first', inplace=True) 
print('Dropped {}'.format(original_len - len(df)))

print('Final number of documents = {}'.format(len(df)))

# 공시가 두번에 걸쳐 발행되었던 케이스, 하나만 잘 남았는지 확인
df[(df['Ticker']=='SWKS')&(df['period_of_report']=='2021-10-01')]

Dropped 50
Final number of documents = 1379


,form_type,CIK,filing_date,period_of_report,texts_only,image_urls,files_url,form_url,Ticker,CompanyName,item1_business,item1a_risk,item7_mda,len_item1
1188,10-K,4127,2021-11-24,2021-10-01,swks-202110010000004127false2021FY--10-010.70....,[],https://www.sec.gov/Archives/edgar/data/4127/0...,https://www.sec.gov/Archives/edgar/data/4127/0...,SWKS,Skyworks,ITEM 1: BUSINESS.5ITEM 1A: RISK FACTORS.1...,ITEM 1A. RISK FACTORS.You should carefully con...,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,32893


In [278]:
for colname, origin_colname in [('len_item1', 'item1_business'), ('len_item1a', 'item1a_risk'), \
                                ('len_item7', 'item7_mda')]:
    df[colname] = df[origin_colname].apply(lambda x: len(x))
    print(origin_colname)
    print('min={}\nMedian={}\nmax={}'.format(df[colname].min(), df[colname].median(), df[colname].max()))
    print('Number of inappropriately processed documents: {}\n'.format(len(df[(df[origin_colname]=='Something went wrong!')])))

item1_business
min=21
Median=40048.0
max=139163
Number of inappropriately processed documents: 83

item1a_risk
min=21
Median=62257.0
max=156849
Number of inappropriately processed documents: 147

item7_mda
min=21
Median=74960.0
max=173278
Number of inappropriately processed documents: 130



In [280]:
df[(df['item1_business']=='Something went wrong!')]['form_url'].values[:5]

array(['https://www.sec.gov/Archives/edgar/data/895421/000089542120000265/msq4201910k.htm',
       'https://www.sec.gov/Archives/edgar/data/1601712/000160171222000053/syf-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/1739940/000173994021000007/ci-20201231.htm',
       'https://www.sec.gov/Archives/edgar/data/713676/000071367622000019/pnc-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/1179929/000117992920000026/moh-12312019x10k.htm'],
      dtype=object)

In [283]:
for origin_colname in ['item1_business', 'item1a_risk', 'item7_mda']:
    print('{},{}'.format(origin_colname, len(df[(df[origin_colname]!='Something went wrong!')])))

item1_business,1296
item1a_risk,1232
item7_mda,1249
